* https://www.kaggle.com/c/fake-news-pair-classification-challenge/data
* https://github.com/fxsjy/jieba

BERT
* https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb#scrollTo=Hcpfl4N2EdOk
* https://github.com/google-research/bert/blob/master/multilingual.md

Gluon-bert
* https://gluon-nlp.mxnet.io/examples/sentence_embedding/bert.html

Embedding options
* https://github.com/Embedding/Chinese-Word-Vectors (Embedding 1)
* https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md (Embedding 2)

Performance
* Using Embedding 1: Performance 0.70640
* Using Embedding 2: Performance 0.68641
* Using BERT: Performance 0.849
* Using BERT_WEIGHTED: Performance 0.85526
* Using BERT_WEIGHTED+SameTID-Agreed: 0.85479
    * Example: df_test[df_test.pred_label != df_test.pred_label2]인 아이들 다 agreed
* Using BERT_WEIGHTED_train100: Performance 0.85634
* Using BERT_ensemble_3models (cn, multi, uncased24): Performance 0.85619
* Using BERT_ensemble_4models_weighted (cn, cnen, multi, uncased24): Performance 0.86283
* CN 80-20 9개 모델 last epoch average: Performance 0.86169
* 4model _ 9model(Last 2 model) average: 0.86414

내일 submission: train / validation을 통해 epoch 최고값 잡고, 여러모델 최종값 merge 

In [2]:
cd /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/bert_repo

/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/bert_repo


In [3]:
import modeling
import optimization
import run_classifier
import tokenization

In [4]:
cd /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook

/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook


In [5]:
import datetime
import glob
import pandas as pd
import numpy as np
import os
import pickle

In [6]:
import tensorflow as tf
import keras

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 162445676780545778
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11043489924649551517
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8796074883659693924
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14992326656
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 623714503544990177
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:86:00.0, compute capability: 7.0"
]


In [8]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)
        all_label_ids.append(feature.label_id)

    def input_fn(params):

        batch_size = 128

        num_examples = len(features)

        # This is for demo purposes and does NOT scale to large data sets. We do
        # not use Dataset.from_generator() because that uses tf.py_func which is
        # not TPU compatible. The right way to load data is with TFRecordReader.
        d = tf.data.Dataset.from_tensor_slices({
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "segment_ids":
                tf.constant(
                    all_segment_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "label_ids":
                tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
        })

        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        return d

    return input_fn

In [9]:
BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12', 'multi_cased_L-12_H-768_A-12', 'uncased_L-24_H-1024_A-16']
['/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_'+model for model in BERT_MODEL_LIST]

['/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_chinese_L-12_H-768_A-12',
 '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_multi_cased_L-12_H-768_A-12',
 '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_uncased_L-24_H-1024_A-16']

In [10]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model

exp_marker = 'crude-ensemble-geomean'
TASK_DATA_DIR = '../data/FakeNews_BERT/FakeNews_BERT/'


# BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12', 'multi_cased_L-12_H-768_A-12', 'uncased_L-24_H-1024_A-16']
BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12']

for BERT_MODEL in BERT_MODEL_LIST:
    for numrun in range(3,15):
        print(str(numrun) + "_!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        train_path = '../data/train.csv'
        df_train = pd.read_csv(train_path)

        df_train['title1_zh'] = df_train.title1_zh.apply(lambda x: str(x).replace('\n', ','))
        df_train['title1_zh'] = df_train.title1_zh.apply(lambda x: str(x).replace('\t', ','))
        df_train['title2_zh'] = df_train.title2_zh.apply(lambda x: str(x).replace('\n', ','))
        df_train['title2_zh'] = df_train.title2_zh.apply(lambda x: str(x).replace('\t', ','))

        idxs = list(df_train.index)
        np.random.shuffle(idxs)
        val_num = len(idxs) // 5 
        train_idxs = idxs[:-val_num] 
        val_idxs = idxs[-val_num:]

        df_tra = df_train.loc[train_idxs, :][['title1_zh', 'title2_zh', 'label']]
        df_val = df_train.loc[val_idxs, :][['title1_zh', 'title2_zh', 'label']]

        df_tra.to_csv('../data/FakeNews_BERT/FakeNews_BERT/train_80_2.tsv', sep='\t',header=True, index=False)
        df_val.to_csv('../data/FakeNews_BERT/FakeNews_BERT/dev_20_2.tsv', sep='\t',header=True, index=False)


        BERT_PRETRAINED_DIR = '../data/' + BERT_MODEL
        OUTPUT_DIR = '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized_all_'+str(numrun)+'_'+BERT_MODEL
        os.mkdir(OUTPUT_DIR)

        print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
        print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

        if BERT_MODEL == 'uncased_L-24_H-1024_A-16':
            TRAIN_BATCH_SIZE = 12
            EVAL_BATCH_SIZE = 12
            TEST_BATCH_SIZE = 12
            LEARNING_RATE = 2e-5
            NUM_TRAIN_EPOCHS = 3.0
            WARMUP_PROPORTION = 0.1
            MAX_SEQ_LENGTH = 43  # According to our dataset, length of 43 can cover 99.5% of the titles.
            TASK = 'FakeEN' 
        else:
            # Model Hyper Parameters
            TRAIN_BATCH_SIZE = 128 # 32
            EVAL_BATCH_SIZE = 128  # 8    
            TEST_BATCH_SIZE = 128  # 8    
            LEARNING_RATE = 2e-5
            NUM_TRAIN_EPOCHS = 5
            WARMUP_PROPORTION = 0.1
            MAX_SEQ_LENGTH = 43  # According to our dataset, length of 43 can cover 99.5% of the titles.
            TASK = 'Fake'

        # Model configs
        SAVE_CHECKPOINTS_STEPS = 1000
        ITERATIONS_PER_LOOP = 1000
        NUM_TPU_CORES = 8
        VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
        CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
        INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
        DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

        processors = {
          "cola": run_classifier.ColaProcessor,
          "mnli": run_classifier.MnliProcessor,
          "mrpc": run_classifier.MrpcProcessor,
          "fake": run_classifier.FakeProcessor,
          "fakeen": run_classifier.FakeENProcessor,  
        }
        processor = processors[TASK.lower()]()
        label_list = processor.get_labels()

        tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
        tpu_cluster_resolver = None

    #     run_config = tf.estimator.RunConfig(
    #         model_dir=OUTPUT_DIR,
    #         save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    #     )

        run_config = tf.contrib.tpu.RunConfig(
            cluster=tpu_cluster_resolver,
            model_dir=OUTPUT_DIR,
            save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
            tpu_config=tf.contrib.tpu.TPUConfig(
                iterations_per_loop=ITERATIONS_PER_LOOP,
                num_shards=NUM_TPU_CORES,
                per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


        train_examples = processor.get_train_examples(TASK_DATA_DIR)
        eval_examples = processor.get_dev_examples(TASK_DATA_DIR)

        num_train_steps = int(
            len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
        num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

        model_fn = run_classifier.model_fn_builder(
            bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
            num_labels=len(label_list),
            init_checkpoint=INIT_CHECKPOINT,
            learning_rate=LEARNING_RATE,
            num_train_steps=num_train_steps,
            num_warmup_steps=num_warmup_steps,
            use_tpu = False,
            use_one_hot_embeddings=True)

        estimator = tf.contrib.tpu.TPUEstimator(
            use_tpu=False,
            model_fn=model_fn,
            config=run_config,
            train_batch_size=TRAIN_BATCH_SIZE,
            eval_batch_size=EVAL_BATCH_SIZE)


        # Train the model.
        print('Feature generating with train data')

        train_features = run_classifier.convert_examples_to_features(
            train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

        eval_features = run_classifier.convert_examples_to_features(
            eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)


        print('***** Started training at {} *****'.format(datetime.datetime.now()))
        print('  Num examples = {}'.format(len(train_examples)))
        print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
        tf.logging.info("  Num steps = %d", num_train_steps)

        train_input_fn = run_classifier.input_fn_builder(
            features=train_features,
            seq_length=MAX_SEQ_LENGTH,
            is_training=True,
            drop_remainder=False)


        eval_input_fn = input_fn_builder(
            features=eval_features,
            seq_length=MAX_SEQ_LENGTH,
            is_training=False,
            drop_remainder=False)


        train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_train_steps)
        eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

        tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

    #     estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    #     print('***** Finished training at {} *****'.format(datetime.datetime.now()))



        # Test the model.
        test_examples = processor.get_test_examples(TASK_DATA_DIR)
        test_features = run_classifier.convert_examples_to_features(
            test_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

        # Eval will be slightly WRONG on the TPU because it will truncate
        # the last batch.
        test_input_fn = input_fn_builder(          # run_classifier.
            features=test_features,
            seq_length=MAX_SEQ_LENGTH,
            is_training=False,
            drop_remainder=False)

        result = estimator.predict(input_fn=test_input_fn)

        raw_result_list = []
        result_list = []
        for rr in result:
            raw_result_list.append(rr)
            result_list.append(np.argmax(rr))


        test_path = '../data/test.csv'
        submission_path = '../data/sample_submission_BERTopt_all_'+str(numrun)+'_'+BERT_MODEL+'.csv'
        submission_all_path = '../data/sample_submission_BERTopt_all_'+str(numrun)+'_'+BERT_MODEL+'_ALL.csv'
        result_raw_path = '../data/sample_submission_sd_BERTopt_all_'+str(numrun)+'_'+BERT_MODEL+'_RESULT_SOFTMAX.p'
        df_test = pd.read_csv(test_path)
        df_test['pred_label'] = result_list
        idx_label = {0:'unrelated', 1:'agreed', 2:'disagreed'}
        df_test['pred_label'] = df_test['pred_label'].apply(lambda x: idx_label[x])

        df_test[['id', 'pred_label']].rename(columns={'id':'Id','pred_label':'Category'}).to_csv(submission_path, index=False)
        df_test.to_csv(submission_all_path, index=False)

        pickle.dump(raw_result_list, open(result_raw_path, 'wb'))

3_!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
***** BERT pretrained directory: ../data/chinese_L-12_H-768_A-12 *****
***** Model output directory: /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized3_chinese_L-12_H-768_A-12 *****
INFO:tensorflow:Using config: {'_model_dir': '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized3_chinese_L-12_H-768_A-12', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe

INFO:tensorflow:global_step/sec: 1.44565
INFO:tensorflow:examples/sec: 185.044
INFO:tensorflow:global_step/sec: 1.52614
INFO:tensorflow:examples/sec: 195.346
INFO:tensorflow:global_step/sec: 1.36134
INFO:tensorflow:examples/sec: 174.252
INFO:tensorflow:global_step/sec: 1.35912
INFO:tensorflow:examples/sec: 173.967
INFO:tensorflow:global_step/sec: 1.35804
INFO:tensorflow:examples/sec: 173.829
INFO:tensorflow:Saving checkpoints for 3000 into /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized3_chinese_L-12_H-768_A-12/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 43)
INFO:tensorflow:  name = input_mask, shape = (?, 43)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 43)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-12-10-19:40:40
INFO:tensorflow:Saving dict for global step 7000: eval_accuracy = 0.892109, eval_loss = 0.0215139, global_step = 7000, loss = 0.0215139
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7000: /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized3_chinese_L-12_H-768_A-12/model.ckpt-7000
INFO:tensorflow:evaluation_loop marked as finished
INFO:tensorflow:global_step/sec: 1.32591
INFO:tensorflow:examples/sec: 169.716
INFO:tensorflow:global_step/sec: 1.77079
I

INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-12-10-20:09:08
INFO:tensorflow:Saving dict for global step 10017: eval_accuracy = 0.895391, eval_loss = 0.021665, global_step = 10017, loss = 0.021665
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10017: /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized3_chinese_L-12_H-768_A-12/model.ckpt-10017
INFO:tensorflow:evaluation_loop marked as finished
INFO:tensorflow:Loss for final step: 0.00906051.
INFO:tensorflow:training_loop marked as finished
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 43)
INFO:tensorflow:  name = input_mask, shape = (?, 43)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 43)
INFO:tensorflow:**** Trainable Variables ***

INFO:tensorflow:examples/sec: 222.814
INFO:tensorflow:global_step/sec: 2.45158
INFO:tensorflow:examples/sec: 313.802
INFO:tensorflow:global_step/sec: 2.67492
INFO:tensorflow:examples/sec: 342.389
INFO:tensorflow:global_step/sec: 2.67136
INFO:tensorflow:examples/sec: 341.934
INFO:tensorflow:global_step/sec: 1.90396
INFO:tensorflow:examples/sec: 243.707
INFO:tensorflow:Saving checkpoints for 3000 into /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized4_chinese_L-12_H-768_A-12/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 43)
INFO:tensorflow:  name = input_mask, shape = (?, 43)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 43)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-10-20:43:07
INFO:tensorflow:Graph was fi

...

INFO:tensorflow:global_step/sec: 2.6924
INFO:tensorflow:examples/sec: 344.627
INFO:tensorflow:global_step/sec: 2.68816
INFO:tensorflow:examples/sec: 344.084
INFO:tensorflow:global_step/sec: 2.68678
INFO:tensorflow:examples/sec: 343.908
INFO:tensorflow:global_step/sec: 2.68511
INFO:tensorflow:examples/sec: 343.695
INFO:tensorflow:global_step/sec: 2.69112
INFO:tensorflow:examples/sec: 344.463
INFO:tensorflow:Saving checkpoints for 6000 into /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized13_chinese_L-12_H-768_A-12/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:tensorflow:global_step/sec: 2.43762
INFO:tensorflow:examples/sec: 312.016
INFO:tensorflow:global_step/sec: 2.69479
INFO:tensorflow:examples/sec: 344.933
INFO:tensorflow:global_step/sec: 2.69189
INFO:tensorflow:examples/sec: 344.562
INFO:tensorflow:global_step/sec: 2.69137
INFO:tensorflow:examples/sec: 344.495
INFO:tensorflow:global_step/sec: 2.69092
INF

INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 43)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-11-09:20:55
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized13_chinese_L-12_H-768_A-12/model.ckpt-10017
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-12-11-09:21:09
INFO:tensorflow:Saving dict for global step 10017: 

INFO:tensorflow:examples/sec: 344.028
INFO:tensorflow:global_step/sec: 2.6911
INFO:tensorflow:examples/sec: 344.461
INFO:tensorflow:Saving checkpoints for 2000 into /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized14_chinese_L-12_H-768_A-12/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:tensorflow:global_step/sec: 2.46182
INFO:tensorflow:examples/sec: 315.113
INFO:tensorflow:global_step/sec: 2.68944
INFO:tensorflow:examples/sec: 344.248
INFO:tensorflow:global_step/sec: 2.68833
INFO:tensorflow:examples/sec: 344.106
INFO:tensorflow:global_step/sec: 2.68539
INFO:tensorflow:examples/sec: 343.73
INFO:tensorflow:global_step/sec: 2.6926
INFO:tensorflow:examples/sec: 344.653
INFO:tensorflow:global_step/sec: 2.69078
INFO:tensorflow:examples/sec: 344.42
INFO:tensorflow:global_step/sec: 2.68907
INFO:tensorflow:examples/sec: 344.201
INFO:tensorflow:global_step/sec: 2.68959
INFO:tensorflow:examples/sec: 344.268
INFO:tens

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 43)
INFO:tensorflow:  name = input_mask, shape = (?, 43)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 43)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-11-10:12:31
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized14_chinese_L-12_H-768_A-12/model.ckpt-7000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorf

INFO:tensorflow:  name = input_mask, shape = (?, 43)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 43)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_optimized14_chinese_L-12_H-768_A-12/model.ckpt-10017
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:prediction_loop marked as finished
INFO:tensorflow:prediction_loop marked as finished


In [ ]:
## Geomean

submission_ensemble_path = '../data/sample_submission_sd_BERT_weightedavg_4model_cn.csv'
smax = {}
BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12', 'chinese_L-12_H-768_A-12_EN', 'multi_cased_L-12_H-768_A-12', 'uncased_L-24_H-1024_A-16']
# BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12']
for BERT_MODEL in BERT_MODEL_LIST: 
    pkls_path = '../data/sample_submission_sd_BERT_'+BERT_MODEL+'_RESULT_SOFTMAX.p'
    with open(pkls_path, 'rb') as file:
        smax[BERT_MODEL] = np.asarray(pickle.load(file))
        
# ens_rslt = np.multiply(smax['chinese_L-12_H-768_A-12', smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16']])   
ens_rslt = np.average(np.array([smax['chinese_L-12_H-768_A-12'], smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16'], smax['chinese_L-12_H-768_A-12_EN']]), 
                      axis=0, weights=[0.6,0.1,0.1,0.2])


df_test = pd.read_csv(test_path)
df_test['pred_label'] = np.argmax(ens_rslt, axis=1)
idx_label = {0:'unrelated', 1:'agreed', 2:'disagreed'}
df_test['pred_label'] = df_test['pred_label'].apply(lambda x: idx_label[x])

df_test[['id', 'pred_label']].rename(columns={'id':'Id','pred_label':'Category'}).to_csv(submission_ensemble_path, index=False)

In [ ]:
ens_rslt = np.average(np.array([smax['chinese_L-12_H-768_A-12'], smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16'], smax['chinese_L-12_H-768_A-12_EN']]), 
                      axis=0, weights=[0.6,0.1,0.1,0.2])

In [ ]:
ens_rslt

In [ ]:
df_test['pred_label'].value_counts()

In [ ]:
df_train = pd.read_csv('../data/train.csv')
dft_tr = df_train['label'].value_counts()
dft_tr/np.sum(dft_tr)

In [ ]:
dft = df_test['pred_label'].value_counts()
dft/np.sum(dft)